# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
# Write your code below.
%load_ext dotenv
%dotenv


In [2]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [3]:
import os
from glob import glob

# Write your code below.
PRICE_DATA = os.getenv("PRICE_DATA")
parquet_files = glob(os.path.join(PRICE_DATA, "**/*.parquet"), recursive=True)

print(f'Number of parquet files found: {len(parquet_files)}')
print('Examples:', *parquet_files[:5], sep='\n')

Number of parquet files found: 3040
Examples:
../../05_src/data/prices/GDX/GDX_2018/part.0.parquet
../../05_src/data/prices/GDX/GDX_2018/part.1.parquet
../../05_src/data/prices/GDX/GDX_2020/part.0.parquet
../../05_src/data/prices/GDX/GDX_2020/part.1.parquet
../../05_src/data/prices/GDX/GDX_2016/part.0.parquet


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [4]:
# Write your code below.
dd_px = dd.read_parquet(parquet_files).set_index('ticker')

dd_feat = dd_px.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(
        Close_lag_1 = x['Close'].shift(1),
        AdjClose_lag_1 = x['Adj Close'].shift(1)
    )
).assign(
    returns = lambda x: x['Close'] / x['Close_lag_1'] - 1,
    hi_lo_range = lambda x: x['High'] - x['Low']
)

dd_feat


/var/folders/f6/y3jng3hj1kl90jtw8jbcxcf40000gn/T/ipykernel_11879/3410490902.py:4: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = dd_px.groupby('ticker', group_keys=False).apply(


,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,AdjClose_lag_1,returns,hi_lo_range
npartitions=90,,,,,,,,,,,,,
A,datetime64[ns],float64,float64,float64,float64,float64,float64,string,int32,float64,float64,float64,float64
ACB,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZEUS,...,...,...,...,...,...,...,...,...,...,...,...,...
ZEUS,...,...,...,...,...,...,...,...,...,...,...,...,...


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [5]:
# Write your code below.
df = dd_feat.compute()

In [6]:
df['returns_MA10'] = df.groupby('ticker')['returns'].rolling(10).mean().reset_index(level=0, drop=True)
df.groupby('ticker').head(20)

,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,AdjClose_lag_1,returns,hi_lo_range,returns_MA10
ticker,,,,,,,,,,,,,,
A,1999-11-18,32.546494,35.765381,28.612303,31.473534,27.068665,62546300.0,A.csv,1999,NaN,NaN,NaN,7.153078,NaN
A,1999-11-19,30.713520,30.758226,28.478184,28.880543,24.838577,15234100.0,A.csv,1999,31.473534,27.068665,-0.082386,2.280043,NaN
A,1999-11-22,29.551144,31.473534,28.657009,31.473534,27.068665,6577800.0,A.csv,1999,28.880543,24.838577,0.089783,2.816525,NaN
A,1999-11-23,30.400572,31.205294,28.612303,28.612303,24.607880,5975600.0,A.csv,1999,31.473534,27.068665,-0.090909,2.592991,NaN
A,1999-11-24,28.701717,29.998211,28.612303,29.372318,25.261524,4843200.0,A.csv,1999,28.612303,24.607880,0.026563,1.385908,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZEUS,2004-01-26,8.030000,8.050000,7.770000,8.000000,7.326987,98700.0,ZEUS.csv,2004,7.900000,7.235398,0.012658,0.280000,0.009064
ZEUS,2004-01-27,8.100000,8.120000,7.600000,7.850000,7.189603,42600.0,ZEUS.csv,2004,8.000000,7.326987,-0.018750,0.520000,0.007599
ZEUS,2004-01-28,7.720000,7.740000,7.390000,7.600000,6.960637,38600.0,ZEUS.csv,2004,7.850000,7.189603,-0.031847,0.350000,0.004414


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

**Answers to the above questions:**

> It was not strictly necessary to convert from dask to pandas dataframe. The same could have been done using the dask dataframe. However, if the dataset is small enough to fit in the memory, then it's simpler and faster to use pandas dataframe. 
> 
> For large datasets that don't fit into memory, it would certainly be better to do it in dask (instead of pandas) because dask provides distributed and lazy computation which can easily scale up to process very large datasets. 

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.